In [1]:
import time

import pandas as pd
import numpy as np
import torch
import pyro

from numpy.linalg import inv

In [2]:
np.version

<module 'numpy.version' from '/Users/nishimura_yuki/workspace/mlpp/pop-pc/pop-pc/lib/python3.7/site-packages/numpy/version.py'>

In [6]:
# Data: https://www.kaggle.com/rounakbanik/the-movies-dataset/download
ratings = pd.read_csv('../data/the-movies-dataset/ratings_small.csv')

In [7]:
# Run this when you haven't created a indexed_ratings.csv yet
unique_userId = ratings.userId.unique()
unique_movieId = ratings.movieId.unique()

ratings['new_user_index'], ratings['new_movie_index'] = 0, 0

for old_id, new_id in zip(unique_userId, range(len(unique_userId))):
    ratings['new_user_index'].iloc[ratings[ratings['userId']==old_id].index.tolist()] = new_id

for old_id, new_id in zip(unique_movieId, range(len(unique_movieId))):
    ratings['new_movie_index'].iloc[ratings[ratings['movieId']==old_id].index.tolist()] = new_id
    
ratings.to_csv('../data/indexed_ratings.csv')

/Users/nishimura_yuki/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
# Run this when you have a indexed_ratings.csv 
ratings = pd.read_csv('../data/indexed_ratings.csv')

In [4]:
# Reference: https://sandipanweb.wordpress.com/2017/04/04/probabilistic-matrix-factorization-with-a-generative-model-in-python/
class PMF:
    def __init__(self, k, prior_mean=0, prior_std=1, epochs=10):
        self.k = k
        self.prior_mean = prior_mean
        self.prior_std = prior_std
        self.epochs = epochs
        self.u = None
        self.v = None
        
    def fit(self, data):
        ratings = data
        self.I = ratings.userId.nunique()
        self.J = ratings.movieId.nunique()
        self.u = pyro.sample("user_factor", pyro.distributions.Normal(self.prior_mean, self.prior_std), sample_shape=torch.Size([self.I,self.k]))
        self.v = pyro.sample("movie_factor", pyro.distributions.Normal(self.prior_mean, self.prior_std), sample_shape=torch.Size([self.J,self.k]))
        self.data_var = ratings['rating'].var(axis=0)
        
        user_to_movie_dict = {}
        movie_to_user_dict = {}

        for i in range(self.I):
            user_to_movie_dict[i] = ratings[ratings['new_user_index']==i]['new_movie_index'].tolist()
        for j in range(self.J):
            movie_to_user_dict[j] = ratings[ratings['new_movie_index']==j]['new_user_index'].tolist()
            
        for epoch_num in range(self.epochs):
            # Update of all u_i: Takes 110.70313000679016 seconds
            for i in range(len(self.u)):
                v_sum = 0
                mv_sum = 0
                for j in user_to_movie_dict[i]:
                    v_sum += np.outer(self.v[j],self.v[j])
                    mv_sum += float(ratings[(ratings['new_user_index']==i) & (ratings['new_movie_index']==j)].rating) * self.v[j]
                self.u[i] = torch.from_numpy(np.dot(inv(self.prior_std * self.data_var * np.identity(self.k) + v_sum), mv_sum))

            # Update of all v_j: Takes 106.21870613098145 seconds
            for j in range(len(self.v)):
                u_sum = 0
                mu_sum = 0
                for i in movie_to_user_dict[j]:
                    u_sum += np.outer(self.u[i],self.u[i])
                    mu_sum += float(ratings[(ratings['new_user_index']==i) & (ratings['new_movie_index']==j)].rating) * self.u[i]
                self.v[j] = torch.from_numpy(np.dot(inv(self.prior_std * self.data_var * np.identity(self.k) + u_sum), mu_sum))

            M_prime = torch.mm(self.u,self.v.T)
            error = 0
            for i in range(len(self.u)):
                for j in user_to_movie_dict[i]:
                    error += (float(ratings[(ratings['new_user_index']==i) & (ratings['new_movie_index']==j)].rating) - M_prime[i][j])**2
            print('MSE of Epoch ' + str(epoch_num) + ': ' + str(float(error/len(ratings))))

In [5]:
# PPC 
# get distribution of ratings from the entire data, and for each user
# get distribution of ratings from the predicted entire data, and predictions for each user
# check if the discrepancies are close to each other

# Discrepancies of original data
max_rating = max(ratings['rating'])
min_rating = min(ratings['rating'])
mean_rating = ratings['rating'].mean()
median_rating = ratings['rating'].median()
first_q_rating = np.quantile(ratings['rating'],0.25)
first_q_rating

AttributeError: module 'numpy' has no attribute 'quantile'

In [6]:
np.version
# POP-PC
# Cross validation POP-PC
# Splitting of train and test data
# Randomly remove ratings from train data, and keep possession of them to check later on 
# Train the model, then compare the entire data distribution and the ratings distributions of each user.

<module 'numpy.version' from '/Users/nishimura_yuki/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/numpy/version.py'>

In [5]:
model = PMF(3)

In [ ]:
model.fit(ratings)

MSE of Epoch 0: 11.04731273651123


In [2]:
print(np.__version__)

1.14.0
